In [4]:
from __future__ import print_function
import matplotlib
matplotlib.use('Agg')
from pixell import enmap, enplot, fft
import numpy as np
import os, sys
sys.path.append("/global/homes/t/toshiyan/Work/Lib/actlib/actsims/")
sys.path.append("/global/homes/t/toshiyan/Work/Lib/actlib/soapack/")
sys.path.append("/global/homes/t/toshiyan/Work/Lib/actlib/")
sys.path.append("/global/homes/t/toshiyan/Work/Lib/actlib/orphics/")
sys.path.append("/global/homes/t/toshiyan/Work/Lib/actlib/tilec/")
from actsims import noise
from soapack import interfaces as sints
from enlib import bench
from orphics import io,stats
import matplotlib.pyplot as plt
from tilec import covtools
import argparse

In [11]:
# Parse command line
version = 'v0'
model = 'dr5'
do_only_filter_noise = False
mask_version = 'padded_v1'
mask_kind = 'binary_apod'
mask_patch = 'boss'
mask_pad = None
extract_mask = None
covsqrt_kind = 'multipow'
rlmin = 300
fill_min = 150
fill_const = True
nsims = 3
radial_fit_annulus = 20
dfact = 8
delta_ell = 100
aminusc = False
no_write = False
calibrated = False
no_off = False
no_prewhiten = False
overwrite = True
debug = False
lmax = None
qid0 = 'boss_d03'

In [12]:
coadd = not(aminusc)

In [13]:
# DR5 s15 daytime
dm = sints.DR5()
season = dm.ainfo(qid0,'season')
patch = dm.ainfo(qid0,'region')
array = dm.ainfo(qid0,'array')
print(array)
#qid = ['boss_d01']
qid = ['boss_d03','boss_d04']

pa3


In [33]:
if dfact == 0: 
    smooth = False
else: 
    smooth = True
    dfact = (dfact,dfact)

In [34]:
# Make version tag
other_keys={'mask_version':mask_version}
for key in other_keys.keys():
    version += ("_"+key+"_"+str(other_keys[key]))
#####

In [35]:
# Get file name convention
pout,cout,sout = noise.get_save_paths(model,version,coadd,season=season,patch=patch,array=array,mkdir=True,overwrite=overwrite,mask_patch=mask_patch)

/global/homes/t/toshiyan/Work/Lib/actlib/actsims/actsims/noise.py:143: UserWarning: Version directory already exists. Overwriting.
  warnings.warn("Version directory already exists. Overwriting.")


In [36]:
# Get mask
mask = sints.get_act_mr3_crosslinked_mask(mask_patch,version=mask_version,kind=mask_kind,season=season,array=array+"_f150",pad=mask_pad)

In [37]:
# Get data model
dm = sints.models[model](region=mask)

In [38]:
# Get a NoiseGen model
if extract_mask is not None:
    emask = sints.get_act_mr3_crosslinked_mask(mask_patch,version=extract_mask,kind=mask_kind,season=season,array=array+"_f150")
    eshape,ewcs = emask.shape,emask.wcs
else:
    emask = mask
ngen = noise.NoiseGen( qid=qid, version=version, model=model, extract_region=emask, ncache=1, verbose=True )

In [39]:
# Get split data
splits = enmap.enmap( [ dm.get_splits( q ) for q in qid ] )
nsplits = 4

/global/homes/t/toshiyan/Work/Lib/actlib/soapack/soapack/interfaces.py:774: UserWarning: An uncalibrated map was requested.
  if not(calibrated) and warn: warnings.warn("An uncalibrated map was requested.")


(3549, 12250) (3, 3549, 12250)
(3549, 12250) (3, 3549, 12250)
(3549, 12250) (3, 3549, 12250)
(3549, 12250) (3, 3549, 12250)


/global/homes/t/toshiyan/Work/Lib/actlib/soapack/soapack/interfaces.py:774: UserWarning: An uncalibrated map was requested.
  if not(calibrated) and warn: warnings.warn("An uncalibrated map was requested.")


(3549, 12250) (3, 3549, 12250)
(3549, 12250) (3, 3549, 12250)
(3549, 12250) (3, 3549, 12250)
(3549, 12250) (3, 3549, 12250)


In [40]:
print(np.shape(splits))
splits.wcs

(2, 4, 3, 3549, 12250)


car:{cdelt:[-0.008333,0.008333],crval:[182.5,0],crpix:[7798,748]}

In [41]:
# Get inverse variance map
ivars = enmap.enmap( [ dm.get_ivars( q ) for q in qid ] )

/global/homes/t/toshiyan/Work/Lib/actlib/soapack/soapack/interfaces.py:780: UserWarning: An uncalibrated map was requested.
  if not(calibrated) and warn: warnings.warn("An uncalibrated map was requested.")


(3549, 12250) (3549, 12250)
(3549, 12250) (3549, 12250)
(3549, 12250) (3549, 12250)
(3549, 12250) (3549, 12250)


/global/homes/t/toshiyan/Work/Lib/actlib/soapack/soapack/interfaces.py:780: UserWarning: An uncalibrated map was requested.
  if not(calibrated) and warn: warnings.warn("An uncalibrated map was requested.")


(3549, 12250) (3549, 12250)
(3549, 12250) (3549, 12250)
(3549, 12250) (3549, 12250)
(3549, 12250) (3549, 12250)


In [42]:
print(np.shape(ivars))
ivars.wcs

(2, 4, 1, 3549, 12250)


car:{cdelt:[-0.008333,0.008333],crval:[182.5,0],crpix:[7798,748]}

In [43]:
modlmap = splits.modlmap()
flatstring = "un" if do_only_filter_noise else ""
n2d_xflat = noise.get_n2d_data(splits,ivars,mask,coadd_estimator=coadd,flattened=not(do_only_filter_noise),plot_fname=pout+"_n2d_%sflat" % flatstring if debug else None,dtype=dm.dtype)
ncomps = n2d_xflat.shape[0]
if ncomps==1: npol = 1
else: npol = 3
mask_ell = rlmin - radial_fit_annulus
#del splits

/global/homes/t/toshiyan/Work/Lib/actlib/actsims/actsims/noise.py:258: RuntimeWarning: invalid value encountered in true_divide
  retmap = np.nan_to_num(np.sum(wts*imaps,axis=axis)/twt)
/global/homes/t/toshiyan/Work/Lib/actlib/actsims/actsims/noise.py:180: RuntimeWarning: divide by zero encountered in true_divide
  sivars   = ((1./ivars) - (1./civars[:,None,...]))**-0.5
/global/homes/t/toshiyan/Work/Lib/actlib/actsims/actsims/noise.py:180: RuntimeWarning: divide by zero encountered in true_divide
  sivars   = ((1./ivars) - (1./civars[:,None,...]))**-0.5
/global/homes/t/toshiyan/Work/Lib/actlib/actsims/actsims/noise.py:180: RuntimeWarning: invalid value encountered in subtract
  sivars   = ((1./ivars) - (1./civars[:,None,...]))**-0.5
/global/homes/t/toshiyan/Work/Lib/actlib/actsims/actsims/noise.py:180: RuntimeWarning: divide by zero encountered in power
  sivars   = ((1./ivars) - (1./civars[:,None,...]))**-0.5


In [ ]:
radial_pairs = [(0,0),(1,1),(2,2),(3,3),(4,4),(5,5),(0,3),(3,0)] if not(no_prewhiten) else []
if smooth:
    n2d_xflat_smoothed = noise.smooth_ps(n2d_xflat.copy(),dfact=dfact,radial_pairs=radial_pairs,
                                        plot_fname=pout+"_n2d_%sflat_smoothed" % flatstring if debug else None,
                                        radial_fit_annulus = radial_fit_annulus,
                                        radial_fit_lmin=rlmin,fill_lmax=lmax,log=not(delta_ell is None),delta_ell=delta_ell,nsplits=nsplits)
else:
    n2d_xflat_smoothed = n2d_xflat.copy()
del n2d_xflat

In [15]:
'''
fill_val = 0
if fill_const: fill_val = n2d_xflat_smoothed[:,:,np.logical_and(modlmap>fill_min,modlmap<(fill_min+radial_fit_annulus))].mean()
if fill_min is not None: n2d_xflat_smoothed[:,:,modlmap<fill_min] = fill_val
n2d_xflat_smoothed[:,:,modlmap<2] = 0
if lmax is not None: n2d_xflat_smoothed[:,:,modlmap>lmax] = 0

if no_off: n2d_xflat_smoothed = noise.null_off_diagonals(n2d_xflat_smoothed)

if do_only_filter_noise:
    ngen.save_filter_noise(n2d_xflat_smoothed,season=season,patch=patch,array=array,coadd=coadd,mask_patch=mask_patch)    
    sys.exit()

covsqrt = noise.get_covsqrt(n2d_xflat_smoothed,covsqrt_kind)
del n2d_xflat_smoothed
ngen.save_covsqrt(covsqrt,season=season,patch=patch,array=array,coadd=coadd,mask_patch=mask_patch)
'''

'\nfill_val = 0\nif fill_const: fill_val = n2d_xflat_smoothed[:,:,np.logical_and(modlmap>fill_min,modlmap<(fill_min+radial_fit_annulus))].mean()\nif fill_min is not None: n2d_xflat_smoothed[:,:,modlmap<fill_min] = fill_val\nn2d_xflat_smoothed[:,:,modlmap<2] = 0\nif lmax is not None: n2d_xflat_smoothed[:,:,modlmap>lmax] = 0\n\nif no_off: n2d_xflat_smoothed = noise.null_off_diagonals(n2d_xflat_smoothed)\n\nif do_only_filter_noise:\n    ngen.save_filter_noise(n2d_xflat_smoothed,season=season,patch=patch,array=array,coadd=coadd,mask_patch=mask_patch)    \n    sys.exit()\n\ncovsqrt = noise.get_covsqrt(n2d_xflat_smoothed,covsqrt_kind)\ndel n2d_xflat_smoothed\nngen.save_covsqrt(covsqrt,season=season,patch=patch,array=array,coadd=coadd,mask_patch=mask_patch)\n'

In [ ]:
if nsims>0:
    bin_edges = np.arange(40,8000,40)
    p1ds = []
    for i in range(nsims):
        print("Sim %d of %d ..." % (i+1,nsims))
        with bench.show("simgen"):
            sims = ngen.generate_sim(season=season,patch=patch,array=array,seed=i,mask_patch=mask_patch)
            print(sims.nbytes/1024./1024./1024., " GB", sims.shape, sims.dtype)
        if extract_mask is not None: 
            ivars2 = enmap.extract(ivars,eshape,ewcs)
            modlmap = enmap.modlmap(eshape,ewcs)
        else:
            ivars2 = ivars

        if debug and i==0: noise.plot(pout+"_sims",sims)
        if not(no_write):
            ngen.save_sims(i,sims,season,patch,array,coadd=coadd,mask_patch=mask_patch)
        n2d_sim = noise.get_n2d_data(sims,ivars2,emask,coadd_estimator=coadd,flattened=False,plot_fname=pout+"_n2d_sim" if (debug and i==0) else None,dtype = dm.dtype)
        del sims
        cents,op1ds_sim = noise.get_p1ds(n2d_sim,modlmap,bin_edges)
        p1ds.append(op1ds_sim.copy().reshape(-1))
    p1dstats = stats.get_stats(np.array(p1ds))

    #del covsqrt

    # For verification
    splits = dm.get_splits(qid)
    splits = np.expand_dims(splits,axis=0)
    
    if extract_mask is not None: 
        splits = enmap.extract(splits,eshape,ewcs)

    n2d_data = noise.get_n2d_data(splits,ivars2,emask,coadd_estimator=coadd,dtype=dm.dtype)
    cents,p1ds_data = noise.get_p1ds(n2d_data,modlmap,bin_edges)
    corr = noise.corrcoef(n2d_data)
    del n2d_data
    bin_edges = np.arange(40,10000,100)
    cents2,c1ds_data = noise.get_p1ds(corr,modlmap,bin_edges)
    noise.plot_corrcoeff(cents2,c1ds_data,plot_fname=pout)

    noise.compare_ps(cents,p1dstats['mean'].reshape((npol,npol,cents.size)),p1ds_data,plot_fname="%s_compare" % (pout),err=p1dstats['errmean'].reshape((npol,npol,cents.size)))
